In [15]:
%pylab
#%pylab inline
#from pylab import *

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [16]:
# PITMAN 1978 MODEL

# Define parameters
R_SS = 2.5*10.**-5 #Subsidence rate in m/yr
P = 10.**8 #Period of sea level oscillations in years
slope = 1./5000.
omega = 2.*pi/P #angular frequency [1/yrs]
D = 2.5*10**5 #meters
Psi = R_SS/(slope*D*omega)

# Define timestep and number of steps
dt = 0.001 #this is t* = t/P, 1/1000th of a period
nsteps = 1500 #1.5 periods of oscillation
x = zeros(nsteps+1)
x[0] = 1./(Psi**2.+1) #0.
t = arange(nsteps+1)*dt

# EULER METHOD
'''
for tstep in arange(nsteps):
    dx_dt = -2.*pi*sin(2.*pi*t[tstep]) - 2.*pi*Psi*x[tstep]
    x[tstep+1] = x[tstep] + dt*dx_dt
'''
# PREDICTOR-CORRECTOR METHOD
for tstep in arange(nsteps):
    dx_dt = -2.*pi*sin(2.*pi*t[tstep]) - 2.*pi*Psi*x[tstep]
    x_pred = x[tstep] + dt*dx_dt
    dx_dt_x_pred = -2.*pi*sin(2.*pi*t[tstep+1]) - 2.*pi*Psi*x_pred
    dx_dt_avg = (dx_dt + dx_dt_x_pred)/2
    x[tstep+1] = x[tstep] + dt*dx_dt_avg

In [17]:
# PLOT

# not full analytical solution, but close
x_true = -1./sqrt(Psi**2. + 1.)*sin(2*pi*t - arctan(1./Psi))
y_sl = 0.1*cos(2.*pi*t)

# Figure
figure()
plot(t,x)
plot(t,x_true, 'k--')
plot(t, y_sl, ':')
legend(['Numerical', 'Analytical', 'Sea Level'])
xlabel("Time")
ylabel("Shoreline Position")

In [31]:
# MIDPOINT METHOD

# Define parameters
R_SS = 2.5*10.**-5 #Subsidence rate in m/yr
P = 10.**8 #Period of sea level oscillations in years
slope = 1./5000.
omega = 2.*pi/P #angular frequency [1/yrs]
D = 2.5*10**5 #meters
Psi = R_SS/(slope*D*omega)

# Functions
def dx_dt(xi, ti, Psi):
    return -2.*pi*sin(2.*pi*ti) - 2.*pi*Psi*xi

# Define timestep and number of steps
dt = 0.03 #this is t* = t/P, 1/1000th of a period
nsteps = 150 #1.5 periods of oscillation
x = zeros(nsteps+1)
x[0] = 1./(Psi**2.+1) #0.
t = arange(nsteps+1)*dt

# 2nd ORDER RUNGAR-KUTTA
for tstep in arange(nsteps):
    k1 = dt*dx_dt(x[tstep], t[tstep], Psi)
    k2 = dt*dx_dt(x[tstep]+0.5*k1, t[tstep]+0.5*dt, Psi)
    x[tstep+1] = x[tstep] + k2

# Define timestep and number of steps
dt = 0.03 #this is t* = t/P, 1/1000th of a period
nsteps = 150 #1.5 periods of oscillation
x4 = zeros(nsteps+1)
x4[0] = 1./(Psi**2.+1) #0.
t = arange(nsteps+1)*dt

# 4th ORDER RUNGAR-KUTTA
for tstep in arange(nsteps):
    k1 = dt*dx_dt(x4[tstep], t[tstep], Psi)
    k2 = dt*dx_dt(x4[tstep]+0.5*k1, t[tstep]+0.5*dt, Psi)
    k3 = dt*dx_dt(x4[tstep]+0.5*k2, t[tstep]+0.5*dt, Psi)
    k4 = dt*dx_dt(x4[tstep]+k3, t[tstep+1], Psi)
    x4[tstep+1] = x4[tstep] + k1/6. + k2/3. + k3/3. +k4/6.

# PLOT

# not full analytical solution, but close
x_true = -1./sqrt(Psi**2. + 1.)*sin(2*pi*t - arctan(1./Psi))
y_sl = 0.1*cos(2.*pi*t)

figure()
plot(t,x)
plot(t,x4)
plot(t,x_true, 'k--')
legend(['Midpoint', '4th Order', 'Analytical']) 
xlabel("Time")
ylabel("Shoreline Position")

In [33]:
from scipy.integrate import odeint

In [34]:
def dx_dt(xi, ti, Psi):
    return -2.*pi*sin(2.*pi*ti) - 2.*pi*Psi*xi

# Ode-int solves similar to 4th order Rungar-Kutta, just easier and faster, but different algorithm
# less work, and better
x_ode = odeint(dx_dt, x[0], t, args=(Psi,))

In [35]:
figure()
plot(t, x_ode)